In [3]:
import sys
sys.path.append('../')

from encoder_module import Encoder
import torch
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = pd.read_csv("dataset.csv")
encoder = Encoder()

/opt/homebrew/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# batch_size = 10000
y_unique = dataset['class'].unique()
y_dict = {'Жилищное строительство': 0,
          'Социальные объекты': 1,
          'Транспортная инфраструктура': 2,
          'Финансовые показатели': 3,
          'Перспективы и планы на будущее': 4}

y_text = dataset['class'] #.iloc[0:batch_size]
y_np = np.array(y_text.apply(lambda x: y_dict[x]).to_list())
y_np.shape

(49775,)

In [29]:
# x_text = dataset['sentence'].iloc[0:5000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('1.npy', X_np)

# x_text = dataset['sentence'].iloc[5000:10000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('2.npy', X_np)

# x_text = dataset['sentence'].iloc[10000:15000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('3.npy', X_np)

# x_text = dataset['sentence'].iloc[15000:20000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('4.npy', X_np)

# x_text = dataset['sentence'].iloc[20000:25000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('5.npy', X_np)

# x_text = dataset['sentence'].iloc[25000:30000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('6.npy', X_np)

# x_text = dataset['sentence'].iloc[30000:35000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('7.npy', X_np)

# x_text = dataset['sentence'].iloc[35000:40000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('8.npy', X_np)

# x_text = dataset['sentence'].iloc[40000:45000]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('9.npy', X_np)

# x_text = dataset['sentence'].iloc[45000:49776]
# X_np = encoder.encode(x_text.to_list()).numpy()
# np.save('10.npy', X_np)
# 
# np1 = np.load('1.npy')
# np2 = np.load('2.npy')
# np3 = np.load('3.npy')
# np4 = np.load('4.npy')
# np5 = np.load('5.npy')
# np6 = np.load('6.npy')
# np7 = np.load('7.npy')
# np8 = np.load('8.npy')
# np9 = np.load('9.npy')
# np10 = np.load('10.npy')

# X_np = np.concatenate((np1, np2, np3, np4, np5, np6, np7, np8, np9, np10), axis=0)
# X_np.shape
# np.save('X_emb.npy', X_np)

X_np = np.load('X_emb.npy')
X_np.shape

In [ ]:
X = torch.from_numpy(X_np).type(torch.float)
y = torch.from_numpy(y_np).type(torch.LongTensor)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=42) #stratify=y
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [ ]:
classificator = torch.nn.Sequential(
    torch.nn.Linear(in_features=768, out_features=2048),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=2048, out_features=2048),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=2048, out_features=2048),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=2048, out_features=2048),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=2048, out_features=5)
)
classificator.to(device)


loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=classificator.parameters(), lr=0.05)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100 
    return acc

In [ ]:
def learn_milticlass_classification_model(cl_model, n_epochs, print_every_epoch=100):
    
    torch.manual_seed(42)

    for epoch in range(n_epochs):
        cl_model.train()

        y_logits = cl_model(X_train)
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
        
        loss = loss_fn(y_logits, y_train) 
        acc = accuracy_fn(y_true=y_train,
                          y_pred=y_pred)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        cl_model.eval()
        with torch.inference_mode():
            test_logits = cl_model(X_test)
            test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
            
            test_loss = loss_fn(test_logits, y_test)
            test_acc = accuracy_fn(y_test,test_pred)

        if epoch % print_every_epoch == 0:
            print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%") 
            
learn_milticlass_classification_model(classificator, 2000, 50)

In [ ]:
torch.save(classificator.state_dict(), "clf.pth")